In [ ]:
using LinearAlgebra
using FFTW
using Plots

In [12]:
# constants
hbar = 1.0545718e-34  # Reduced Planck constant
e = 1.6e-19  # Electron charge
epsilon = 10  # Dielectric constant
v_F = 1.0 # Dirac velocity
a = 1 # lattice constant

1

In [13]:
G1 = [2*pi/a, -2*pi/(sqrt(3) * a)]
G2 = [0, 2*pi/a * 2/sqrt(3)]
kappa_1 = (2/3) * G1 + (1/3) * G2
kappa_3 = kappa_1 - (G1 + G2)
kappa_5 = kappa_1 + G1

2-element Vector{Float64}:
 10.471975511965976
 -3.6275987284684357

In [1]:
# 2x2 Pauli X
function sigma_x()
    return [0 1; 1 0]
end
# 2x2 Pauli Y
function sigma_y()
    return [0 -im; im 0]
end
# 2x2 Pauli Z
function sigma_z()
    return [1 0; 0 -1]
end

sigma_z (generic function with 1 method)

In [2]:
# Kinetic term in spin-space
function H_0(kx, ky, hbar, vF)
    return hbar * vF * (kx * sigma_y() - ky * sigma_x())
end

H_0 (generic function with 1 method)

In [10]:
# Kinetic/volume for symmetric phase
function K1(hbar, vF, lambda)
    return -hbar * vF * lambda^3 / (6 * pi)
end

K1 (generic function with 1 method)

In [11]:
# Kinetic/volume for time-reversal breaking phase
function K2(hbar, vF, lambda1, lambda)
    return -hbar * vF * (lambda1^2 + lambda^2)^(3/2) / (6 * pi)
end

K2 (generic function with 1 method)

In [20]:
# Going to make a crude approximation that potential at K-points >> 
# potential everyhere else
function ham_3(kx, ky, hbar, vF, lambda1, lambda2, k1, k2, k3)
    h1 = H_0(kx + k1[1], ky + k1[2], hbar, vF)
    h2 = H_0(kx + k2[1], ky + k2[2], hbar, vF)
    h3 = H_0(kx + k3[1], ky + k3[2], hbar, vF)
    H_temp = kron(h1, [1 0 0; 0 0 0; 0 0 0]) + kron(h2, [0 0 0; 0 1 0; 0 0 0]) + 
    kron(h3, [0 0 0; 0 0 0; 0 0 1])
    H_temp += kron(lambda1 * sigma_z(), [1 0 0; 0 1 0; 0 0 1])
    # periodic potential
    h_connect = [0 lambda2 lambda2; conj(lambda2) 0 lambda2; 
    conj(lambda2) conj(lambda2) 0]
    H_temp += kron([1 0; 0 1], h_connect)
    return H_temp
end

ham_3 (generic function with 1 method)